This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.auxiliary_property_based import *
from graph_world.self_supervised_learning.pretext_tasks.contrastive_based_different_scale import *
from graph_world.self_supervised_learning.pretext_tasks.hybrid import *

In [4]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    "embedding_corruption_ratio" : 0.1, 
    "partial_embedding_reconstruction" : True,
    'n_parts': 10,
    'shortest_path_cutoff': 6,
    'N_classes': 4,
    'k_largest': 20,
    'k': 20,
    'temperature': 1,
    'num_cluster_iter': 500,
    'alpha': 0.15,
    'n_clusters': 30,
    'num_iter': 10
}

generator_config = {
    'num_clusters' : 7,
}

pretext_task = HuEtAL

In [5]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]
# dataset = KarateClub()[0]


In [6]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_task=pretext_task)
                
# benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=~dataset.train_mask, test_mask=~dataset.train_mask)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


TypeError: graph_world.self_supervised_learning.pretext_tasks.generation_based.DenoisingLinkReconstruction is not a Module subclass

In [13]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, Batch
from torch import tensor
from torch_geometric.nn import GraphConv, global_mean_pool

X = torch.tensor([
    [0],
    [1],
    [2],
    [3],
    [4]
], dtype=torch.float32)
edge_index_s = torch.tensor([
    [0, 0, 0, 0],
    [1, 2, 3, 4],
])
edge_index_t = torch.tensor([
    [0, 0, 0],
    [1, 2, 3],
])


a = Data(x=X[0:2], edge_index=edge_index_s) # [0, 1]
b = Data(x=X[1:], edge_index=edge_index_t) # [1, 2, 3, 4]




batch = Batch.from_data_list([a, b])
model = GraphConv(in_channels=1, out_channels=1)
H = model(x=batch.x, edge_index=batch.edge_index)
batch.batchh

AttributeError: 'GlobalStorage' object has no attribute 'batchh'

In [10]:
from torch_geometric.utils import k_hop_subgraph

k_hop_subgraph([0], edge_index=dataset.edge_index, num_hops=1, relabel_nodes=True)[0:2]

(tensor([   0,  633, 1862, 2582]),
 tensor([[0, 0, 0, 1, 2, 2, 3, 3],
         [1, 2, 3, 0, 0, 3, 0, 2]]))